## Deliverable 2. Create a Customer Travel Destinations Map.

In [7]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import sys

sys.path.insert(0, '..')

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [10]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Norman Wells,CA,65.2820,-126.8329,25.12,96,10,3.91,clear sky
1,1,Coahuayana,MX,18.7333,-103.6833,78.33,88,45,2.53,scattered clouds
2,2,Isangel,VU,-19.5500,169.2667,76.42,73,8,14.12,clear sky
3,3,Kapaa,US,22.0752,-159.3190,82.38,72,20,4.00,few clouds
4,4,Mataura,NZ,-46.1927,168.8643,66.47,40,67,3.22,broken clouds


In [11]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 88


In [12]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Coahuayana,MX,18.7333,-103.6833,78.33,88,45,2.53,scattered clouds
2,2,Isangel,VU,-19.5500,169.2667,76.42,73,8,14.12,clear sky
3,3,Kapaa,US,22.0752,-159.3190,82.38,72,20,4.00,few clouds
13,13,Meulaboh,ID,4.1363,96.1285,84.15,67,96,3.18,overcast clouds
17,17,Rikitea,PF,-23.1203,-134.9692,74.07,78,100,16.80,overcast clouds
23,23,Georgetown,MY,5.4112,100.3354,84.88,84,40,3.00,scattered clouds
26,26,Mundo Nuevo,MX,18.7000,-95.5000,80.10,71,31,13.00,scattered clouds
32,32,Ca Mau,VN,9.1769,105.1500,80.42,90,100,9.82,overcast clouds
37,37,Bambous Virieux,MU,-20.3428,57.7575,73.65,83,20,5.75,few clouds
38,38,Bac Lieu,VN,9.2850,105.7244,79.07,89,100,12.01,overcast clouds


In [13]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                264
City                   264
Country                264
Lat                    264
Lng                    264
Max Temp               264
Humidity               264
Cloudiness             264
Wind Speed             264
Current Description    264
dtype: int64

In [14]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

clean_df.count()

City_ID                264
City                   264
Country                264
Lat                    264
Lng                    264
Max Temp               264
Humidity               264
Cloudiness             264
Wind Speed             264
Current Description    264
dtype: int64

In [15]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Coahuayana,MX,78.33,scattered clouds,18.7333,-103.6833,
2,Isangel,VU,76.42,clear sky,-19.5500,169.2667,
3,Kapaa,US,82.38,few clouds,22.0752,-159.3190,
13,Meulaboh,ID,84.15,overcast clouds,4.1363,96.1285,
17,Rikitea,PF,74.07,overcast clouds,-23.1203,-134.9692,
23,Georgetown,MY,84.88,scattered clouds,5.4112,100.3354,
26,Mundo Nuevo,MX,80.10,scattered clouds,18.7000,-95.5000,
32,Ca Mau,VN,80.42,overcast clouds,9.1769,105.1500,
37,Bambous Virieux,MU,73.65,few clouds,-20.3428,57.7575,
38,Bac Lieu,VN,79.07,overcast clouds,9.2850,105.7244,


In [16]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        
hotel_df.head(10)

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Coahuayana,MX,78.33,scattered clouds,18.7333,-103.6833,El rranchito michuacan de ocampos
2,Isangel,VU,76.42,clear sky,-19.5500,169.2667,Tanna Lodge
3,Kapaa,US,82.38,few clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
13,Meulaboh,ID,84.15,overcast clouds,4.1363,96.1285,Kost Pria
17,Rikitea,PF,74.07,overcast clouds,-23.1203,-134.9692,People ThankYou
23,Georgetown,MY,84.88,scattered clouds,5.4112,100.3354,Cititel Penang
26,Mundo Nuevo,MX,80.10,scattered clouds,18.7000,-95.5000,
32,Ca Mau,VN,80.42,overcast clouds,9.1769,105.1500,Khách Sạn Mường Thanh Luxury Cà Mau
37,Bambous Virieux,MU,73.65,few clouds,-20.3428,57.7575,Casa Tia Villa
38,Bac Lieu,VN,79.07,overcast clouds,9.2850,105.7244,Khách sạn Sài Gòn Bạc Liêu


In [33]:
# 7. Drop the rows where there is no Hotel Name.
nan_value = float("NaN")
hotel_df['Hotel Name'].replace('', nan_value, inplace=True)
hotel_df.dropna(subset = ["Hotel Name"], inplace=True)

clean_hotel_df = hotel_df
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Coahuayana,MX,78.33,scattered clouds,18.7333,-103.6833,El rranchito michuacan de ocampos
2,Isangel,VU,76.42,clear sky,-19.5500,169.2667,Tanna Lodge
3,Kapaa,US,82.38,few clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
13,Meulaboh,ID,84.15,overcast clouds,4.1363,96.1285,Kost Pria
17,Rikitea,PF,74.07,overcast clouds,-23.1203,-134.9692,People ThankYou
23,Georgetown,MY,84.88,scattered clouds,5.4112,100.3354,Cititel Penang
32,Ca Mau,VN,80.42,overcast clouds,9.1769,105.1500,Khách Sạn Mường Thanh Luxury Cà Mau
37,Bambous Virieux,MU,73.65,few clouds,-20.3428,57.7575,Casa Tia Villa
38,Bac Lieu,VN,79.07,overcast clouds,9.2850,105.7244,Khách sạn Sài Gòn Bạc Liêu
41,Souillac,MU,73.78,few clouds,-20.5167,57.5167,Shanti Maurice Resort & Spa


In [34]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [37]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [38]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))